In [21]:

"""
This file is containing a class, that perform all modules for 
Hovagim Bakardjian system that serves for feature extraction and command
classification in SSVEP based BCI. 
Also, it has an built-in FFT features extractor

Version 3
"""

import numpy as np
import pandas as pd
from bieg import ICAManager
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.stats import mode
from sklearn.cross_decomposition import CCA


class BakardjianSystem(object):
    
    def __init__(self,path,seconds=1,components_to_exclude = [0,14,27,28,127],
                 chunk = 1,extract = False,freq = 256,classes = 'three_classes',normalize=True):
        self.self = self
        self.path = path
        self.seconds = seconds
        self.ica_file = ICAManager(input_path=self.path,method='fastica',sep=' ')
        self.components_to_exclude = components_to_exclude
        self.extract = extract
        self.freq = freq
        self.classes = classes
        self.normalize = normalize
        
    def prep_data_extract_components(self):
        
        self.ica_file.load_data()
        
        if self.extract == True:
            self.ica_file.extract_components()
            self.ica_file.exclude_ica_components(
                components_to_exclude=self.components_to_exclude)
        
        self.data = self.ica_file.data
        self.o1 = self.data[15]
        self.oz = self.data[23]
        self.o2 = self.data[28]

    @staticmethod
    def filtering(data,low,high,freq):
        
        bplowcut = low/(freq*0.5)
        bphighcut = high/(freq*0.5) 

        [b,a] = sig.butter(N=4,Wn=[bplowcut,bphighcut],btype='bandpass')

        filtered = sig.filtfilt(b,a,data)
        return filtered
    
    def go(self,low,high):
        
        """This functon replaces variance analyzer and smoother, serves as both"""
        
        electrode = np.zeros(self.o1.shape[0])
        
        for n in [self.o1,self.oz,self.o2]:
            filtered = self.filtering(n,low,high,self.freq)
            var = np.abs(filtered)
            smoothed = sig.savgol_filter(var,polyorder=2,
                                         window_length=self.seconds)
            electrode += smoothed
            
        electrode = electrode/3
        
        return electrode
            
    def normalizer(self):
        
        band8hz = [7.9,8.1]
        band14hz = [13.9,14.1]
        band28hz = [27.9,28.1]
        
        if self.classes == 'two_classes':
            
            sig_8hz = self.go(band8hz[0],band8hz[1])
            sig_14hz = self.go(band14hz[0],band14hz[1])
            
            if self.normalize == True:
                self.sig_8hz = sig_8hz/(sig_8hz+sig_14hz)
                self.sig_14hz = sig_14hz/(sig_8hz+sig_14hz)
            else:
                self.sig_8hz = sig_8hz
                self.sig_14hz = sig_14hz
            
        elif self.classes == 'three_classes':
            
            sig_8hz = self.go(band8hz[0],band8hz[1])
            sig_14hz = self.go(band14hz[0],band14hz[1])
            sig_28hz = self.go(band28hz[0],band28hz[1])
            
            if self.normalize == True:
                self.sig_8hz = sig_8hz/(sig_8hz+sig_14hz+sig_28hz)
                self.sig_14hz = sig_14hz/(sig_8hz+sig_14hz+sig_28hz)
                self.sig_28hz = sig_28hz/(sig_8hz+sig_14hz+sig_28hz)
            else:
                self.sig_8hz = sig_8hz
                self.sig_14hz = sig_14hz
                self.sig_28hz = sig_28hz
            
    def classifier(self):
        
        classified = np.zeros(self.sig_8hz.shape)
        dict_classes = dict()
        
        if self.classes == 'two_classes':
            for n in range(self.sig_8hz.shape[0]):
                dict_classes[self.sig_8hz[n]] = 0
                
                dict_classes[self.sig_14hz[n]] = 1

                
                val_max = np.max(np.array([self.sig_8hz[n],self.sig_14hz[n]]))
                classified[n] = val_max
        
        elif self.classes == 'three_classes':
            for n in range(self.sig_8hz.shape[0]):
                dict_classes[self.sig_8hz[n]] = 0
                dict_classes[self.sig_14hz[n]] = 0
                dict_classes[self.sig_28hz[n]] = 1
                val_max = np.max(np.array([self.sig_8hz[n],self.sig_14hz[n]]))
                classified[n] = val_max
        
        decision = dict_classes[np.max(classified)]
        
        return decision 
        
    def extract_FFT(self):
        
        if self.classes == 'two_classes':
            array_of_freq = np.zeros(((3,2,3)))
            for t,p in zip([self.o1,self.oz,self.o2],range(3)):
                result = 2*abs(np.fft.fft(sig.hamming(len(t))*t,self.freq))/self.freq

                for n,z in zip([[7,8,9],[13,14,15]],range(2)):
                    for r in range(3):
                        array_of_freq[p][z][r] = result[n[r]]
                        
            array_of_freq_max =  np.zeros((3,2))
            for z in range(3):
                for n in range(2):
                    array_of_freq_max[z][n] = np.max(array_of_freq[z][n])
            
            features = np.array([np.max(array_of_freq_max[:,[0]]),
                                 np.max(array_of_freq_max[:,[1]])])
                    
            self.features = features
        
        if self.classes == 'three_classes':
            array_of_freq = np.zeros(((3,3,3)))
            for t,p in zip([self.o1,self.oz,self.o2],range(3)):
                result = 2*abs(np.fft.fft(sig.hamming(len(t))*t,self.freq))/self.freq

                for n,z in zip([[7,8,9],[13,14,15],[27,28,29]],range(3)):
                    for r in range(3):
                        array_of_freq[p][z][r] = result[n[r]]
                        
            array_of_freq_max =  np.zeros((3,3))
            for z in range(3):
                for n in range(3):
                    array_of_freq_max[z][n] = np.max(array_of_freq[z][n])
            
            features = np.array([np.max(array_of_freq_max[:,[0]]),
                                 np.max(array_of_freq_max[:,[1]]),
                                 np.max(array_of_freq_max[:,[2]])])
                    
            self.features = features
            
if __name__ == '__main__':
    bs = BakardjianSystem(path= 'subject1/sd28Hz5sec/28Hz5sec2prt2trial.csv',seconds=3,classes='three_classes')
    bs.prep_data_extract_components()
    plt.show()
    bs.normalizer()
    print(bs.classifier())
    bs.extract_FFT()
    print(bs.features)

0
[ 362.8232871   196.60520018   74.64126747]
